# 🌾 Plant Disease Detection System
## Multi-Crop Disease Classification using Deep Learning

**Dataset**: Comprehensive plant disease dataset  
**Model**: MobileNetV2 with transfer learning  
**Output**: Trained model (.h5), TFLite model, evaluation metrics


In [1]:
# OPTIMIZED CPU TRAINING SETUP
print("=== OPTIMIZED CPU TRAINING SETUP ===")
print("🚀 Using yolov8-gpu environment for optimized training!")
print("✅ Python 3.13 detected - Using optimized TensorFlow CPU!")
print("⚡ CPU training optimized for speed!")

# Check if dependencies are available
import subprocess
import sys

print("\nChecking dependencies...")
dependencies = ['tensorflow', 'matplotlib', 'scikit-learn', 'opencv-python', 'seaborn', 'numpy', 'pandas', 'pillow']
missing_deps = []

for dep in dependencies:
    try:
        __import__(dep.replace('-', '_'))
        print(f"✅ {dep} available!")
    except ImportError:
        missing_deps.append(dep)
        print(f"❌ {dep} missing")

if missing_deps:
    print(f"\nInstalling missing dependencies: {missing_deps}")
    for dep in missing_deps:
        try:
            subprocess.run([sys.executable, '-m', 'pip', 'install', dep], check=True)
            print(f"✅ {dep} installed!")
        except Exception as e:
            print(f"❌ {dep} installation failed: {e}")

print("\n✅ Dependencies ready!")
print("⚡ Optimized CPU training enabled!")


=== OPTIMIZED CPU TRAINING SETUP ===
🚀 Using yolov8-gpu environment for optimized training!
✅ Python 3.13 detected - Using optimized TensorFlow CPU!
⚡ CPU training optimized for speed!

Checking dependencies...



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\tusha\anaconda3\envs\plant_gpu\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\tusha\anaconda3\envs\plant_gpu\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\tusha\anaconda3\envs\plant_gpu\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\tusha\anaconda3\envs\plant_gpu\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
 

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\tusha\anaconda3\envs\plant_gpu\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\tusha\anaconda3\envs\plant_gpu\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\tusha\anaconda3\envs\plant_gpu\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\tusha\anaconda3\envs\plant_gpu\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
 

AttributeError: _ARRAY_API not found

...


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\tusha\anaconda3\envs\plant_gpu\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\tusha\anaconda3\envs\plant_gpu\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\tusha\anaconda3\envs\plant_gpu\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\tusha\anaconda3\envs\plant_gpu\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
 

AttributeError: _ARRAY_API not found

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [1]:
# CONFIGURATION - LOCAL GPU TRAINING
import tensorflow as tf
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix

print("=== LOCAL GPU CONFIGURATION ===")
print(f"TensorFlow version: {tf.__version__}")

# Check for GPU
gpus = tf.config.list_physical_devices('GPU')
print(f"Available GPUs: {len(gpus)}")

if gpus:
    print("✅ GPU(s) detected!")
    for i, gpu in enumerate(gpus):
        print(f"  GPU {i}: {gpu}")
    
    # Configure GPU memory growth
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✓ GPU memory growth enabled")
    except RuntimeError as e:
        print(f"GPU memory growth configuration failed: {e}")
    
    # Enable mixed precision for faster training
    try:
        policy = tf.keras.mixed_precision.Policy('mixed_float16')
        tf.keras.mixed_precision.set_global_policy(policy)
        print("✓ Mixed precision training enabled (float16)")
    except Exception as e:
        print(f"Mixed precision setup failed: {e}")
    
    # Test GPU functionality
(plant_gpu) C:\Users\tusha\OneDrive\Desktop\sih>python -m ipykernel install --user --name plant_gpu --display-name "plant_gpu (Python 3.10)"
Installed kernelspec plant_gpu in C:\Users\tusha\AppData\Roaming\jupyter\kernels\plant_gpu

    with tf.device('/GPU:0'):
        test_tensor = tf.constant([1.0, 2.0, 3.0])
        print(f"✓ Test tensor created on GPU: {test_tensor.device}")
    
    print("🚀 Local GPU training is ready!")
    use_gpu = True
    
else:
    print("🖥️  No GPU detected - using CPU training")
    use_gpu = False

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)
print("✅ Local GPU setup complete!")


SyntaxError: invalid syntax (3944026969.py, line 48)

In [2]:
# DATASET SETUP``
os.makedirs('models', exist_ok=True)
data_path = 'final_dataset'


print("=== DATASET EXPLORATION ===")
for split in ['train', 'val', 'test']:
    split_path = os.path.join(data_path, split)
    if os.path.exists(split_path):
        classes = os.listdir(split_path)
        total_images = 0
        for class_name in classes:
            class_path = os.path.join(split_path, class_name)
            if os.path.isdir(class_path):
                image_files = []
                for ext in ['*.jpg', '*.jpeg', '*.JPG', '*.JPEG', '*.png', '*.PNG']:
                    image_files.extend(glob.glob(os.path.join(class_path, ext)))
                total_images += len(image_files)
        print(f"{split.upper()}: {len(classes)} classes, {total_images} images")

print("✅ Dataset ready!")


NameError: name 'os' is not defined

In [ ]:
# DATA PREPROCESSING
img_size = 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2]
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    os.path.join(data_path, 'train'),
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    os.path.join(data_path, 'val'),
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

steps_per_epoch = train_gen.samples // batch_size
validation_steps = val_gen.samples // batch_size

print("Data generators created")
print(f"Classes: {train_gen.num_classes}")
print(f"Training samples: {train_gen.samples}")
print(f"Validation samples: {val_gen.samples}")


In [ ]:
# BUILD MODEL
if use_gpu:
    with tf.device('/GPU:0'):
        base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
        base.trainable = False
        
        x = GlobalAveragePooling2D()(base.output)
        x = Dropout(0.3)(x)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        
        if tf.keras.mixed_precision.global_policy().name == 'mixed_float16':
            x = tf.keras.layers.Activation('linear', dtype='float32')(x)
        
        out = Dense(train_gen.num_classes, activation='softmax', dtype='float32')(x)
        model = Model(base.input, out)
        
        optimizer = tf.keras.mixed_precision.LossScaleOptimizer(Adam(learning_rate=1e-4))
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
else:
    # CPU training
    base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
    base.trainable = False
    
    x = GlobalAveragePooling2D()(base.output)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    out = Dense(train_gen.num_classes, activation='softmax')(x)
    model = Model(base.input, out)
    
    optimizer = Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print(f"Model built: {model.count_params():,} parameters")
model.summary()


In [ ]:
# TRAIN MODEL
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7),
    ModelCheckpoint('models/best_model.h5', monitor='val_accuracy', save_best_only=True)
]

if use_gpu:
    print("Starting GPU training...")
    with tf.device('/GPU:0'):
        history = model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=20,
            steps_per_epoch=steps_per_epoch,
            validation_steps=validation_steps,
            callbacks=callbacks,
            verbose=1
        )
else:
    print("Starting CPU training...")
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=20,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        callbacks=callbacks,
        verbose=1
    )

model.save('models/final_model.h5')
print("Training completed!")


In [ ]:
# TRAINING RESULTS
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

print(f"Final accuracy: {max(history.history['val_accuracy']):.4f}")


In [ ]:
# MODEL EVALUATION
val_gen.reset()
preds = model.predict(val_gen, verbose=1)
y_pred = np.argmax(preds, axis=1)
y_true = val_gen.classes

print("=== CLASSIFICATION REPORT ===")
class_names = list(val_gen.class_indices.keys())
print(classification_report(y_true, y_pred, target_names=class_names))

plt.figure(figsize=(12, 10))
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

accuracy = np.mean(y_pred == y_true)
print(f"Overall Accuracy: {accuracy:.4f}")


In [ ]:
# TFLite Conversion
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open('models/model.tflite', 'wb') as f:
    f.write(tflite_model)

print(f"TFLite model saved: {len(tflite_model) / 1024:.2f} KB")
print("Ready for mobile deployment!")
